How to Load the dataset?

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('conll2003')

1. First letter cap?
2. Length of word
3. Is a stop word?
4. Number of capital letters
5. Number of dots
6. Number of Hyphens
7. Number of digits
8. Number of letters
9. Pos tag(in numerical form)
10. Is a proper noun?
11. Is a verb?
12. Is a cardinal number?
13. Preceeded by Named Entity?
14. Preceeded by article?
15. Is the first word?

In [ ]:
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]


In [ ]:
def get_vector(word,word_vectors,pos,prev_NE,prev_art,is_first):
  vector=[]
  vector.append(1 if word[0].upper()==word[0] else 0) #1. First letter cap?
  vector.append(len(word))#2. Length of word
  vector.append(1 if word in stopwords else 0)#3. Is a stop word?
  vector.append(sum(1 for elem in word if elem.isupper()))#4. Number of capital letters
  vector.append(sum(1 for elem in word if elem=='.'))# 5. Number of dots
  vector.append(sum(1 for elem in word if elem=='-'))#6. Number of Hyphens
  vector.append((sum(1 for elem in word if elem.isnumeric()))/len(word))#7. Number of digits
  vector.append((sum(1 for elem in word if elem.isalpha()))/len(word)) #8. Number of letters
  vector.append(pos)#9. Pos tag(in numerical form)
  if pos ==22:#10. Is a proper noun?
    vector.append(1)
  else:
    vector.append(0)
  if pos >=37 and pos<=42:# 11. Is a verb?
    vector.append(1)
  else:
    vector.append(0) 
  if pos ==11:  #12. Is a cardinal number?
    vector.append(1) 
  else:
    vector.append(0)
  vector.append(prev_NE)#13. Preceeded by Named Entity?
  vector.append(prev_art)#14. Preceeded by article?
  vector.append(is_first)#15. Is the first word?
  word_vectors[word]=vector
  return word_vectors[word]


In [ ]:
word_vectors={}
train_X=[]
train_y=[]

In [ ]:
for sentence,ner_sentence, pos_sentence in zip(dataset['train'].data['tokens'],dataset['train'].data['ner_tags'],dataset['train'].data['pos_tags']):
                                                                   prev_NE=0
                                                                   prev_art=0
                                                                   is_first=1
                                                                   location=0
                                                                   for word,ner_word,pos_word in zip(sentence,ner_sentence,pos_sentence):
                                                                     location+=1
                                                                     train_X.append(get_vector(word.as_py(),word_vectors,pos_word.as_py(),prev_NE,prev_art,is_first))
                                                                     train_y.append(0 if ner_word.as_py()==0 else 1)
                                                                     prev_NE=ner_word.as_py()
                                                                     if pos_word.as_py()==12:
                                                                       prev_art=1
                                                                     else:
                                                                       prev_art=0
                                                                     is_first=0
                                                                    

In [ ]:
from sklearn import svm
clf = svm.SVC(cache_size=10000,C=0.1,gamma=10,class_weight="balanced")

clf.fit(train_X, train_y) 

# Accuracy on Test set

In [ ]:
test_X=[]
test_y=[]


In [ ]:
for sentence,ner_sentence, pos_sentence in zip(dataset['test'].data['tokens'],dataset['test'].data['ner_tags'],dataset['test'].data['pos_tags']):
                                                                   prev_NE=0
                                                                   prev_art=0
                                                                   is_first=1
                                                                   location=0
                                                                   for word,ner_word,pos_word in zip(sentence,ner_sentence,pos_sentence):
                                                                     location+=1
                                                                     test_X.append(get_vector(word.as_py(),word_vectors,pos_word.as_py(),prev_NE,prev_art,is_first))
                                                                     test_y.append(0 if ner_word.as_py()==0 else 1)
                                                                     prev_NE=ner_word.as_py()
                                                                     if pos_word.as_py()==12:
                                                                       prev_art=1
                                                                     else:
                                                                       prev_art=0
                                                                     is_first=0

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

predictions= clf.predict(test_X)
print(classification_report(test_y, predictions,digits=4))
cf_matrix = confusion_matrix(test_y, predictions)
sns.heatmap(cf_matrix, annot=True, fmt='g')